In [1]:
import pandas

In [2]:
import numpy as np
import sys
pandas.set_option('chained_assignment',None)
files = ["ap_2010.csv", "class_size.csv", "demographics.csv", "graduation.csv", "hs_directory.csv", "math_test_results.csv", "sat_results.csv"]
full_data = {}
for f in files:
    d = pandas.read_csv("data/{0}".format(f))
    full_data[f.replace(".csv", "")] = d

In [3]:
full_data["hs_directory"]["DBN"] = full_data["hs_directory"]["dbn"]
full_data["class_size"]["DBN"] = full_data["class_size"].apply(lambda x: "{0:02d}{1}".format(x["CSD"], x["SCHOOL CODE"]), axis=1)
survey1 = pandas.read_csv("data/survey_all.txt", delimiter="\t", encoding='windows-1252')
survey2 = pandas.read_csv("data/survey_d75.txt", delimiter="\t", encoding='windows-1252')
survey1["d75"] = False
survey2["d75"] = True
survey = pandas.concat([survey1, survey2], axis=0, sort=True)

In [4]:
survey["DBN"] = survey["dbn"]
survey_fields = ["DBN", "rr_s", "rr_t", "rr_p", "N_s", "N_t", "N_p", "saf_p_11", "com_p_11", "eng_p_11", "aca_p_11", "saf_t_11", "com_t_11", "eng_t_11", "aca_t_11", "saf_s_11", "com_s_11", "eng_s_11", "aca_s_11", "saf_tot_11", "com_tot_11", "eng_tot_11", "aca_tot_11"]
full_data["survey"] = survey
survey = survey.loc[:,survey_fields]
full_data["survey"] = survey
survey.shape

(1702, 23)

In [5]:
class_size = full_data["class_size"]
class_size = class_size[class_size["GRADE "] == "09-12"]
class_size = class_size[class_size["PROGRAM TYPE"] == "GEN ED"]
class_size = class_size.groupby("DBN").agg(np.mean)
class_size.reset_index(inplace=True)
full_data["class_size"] = class_size
demographics = full_data["demographics"]
demographics = demographics[demographics["schoolyear"] == 20112012]
full_data["demographics"] = demographics
full_data["math_test_results"] = full_data["math_test_results"][full_data["math_test_results"]["Year"] == 2011]
full_data["math_test_results"] = full_data["math_test_results"][full_data["math_test_results"]["Grade"] == '8']
full_data["graduation"] = full_data["graduation"][full_data["graduation"]["Cohort Year"] == 2009]
full_data["graduation"] = full_data["graduation"][full_data["graduation"]["Category"] == "All Students"]
full_data["graduation"] = full_data["graduation"][full_data["graduation"]["Cohort"] == "4 year June"]

In [6]:
full_data["sat_results"].dtypes

DBN                                object
SCHOOL NAME                        object
Num of SAT Test Takers             object
SAT Critical Reading Avg. Score    object
SAT Math Avg. Score                object
SAT Writing Avg. Score             object
dtype: object

In [7]:
full_data["sat_results"] = full_data["sat_results"][full_data["sat_results"]["Num of SAT Test Takers"] != 's']
full_data['sat_results'].reset_index(drop=True, inplace=True)
for c in full_data["sat_results"].columns[3:]:
    full_data["sat_results"][c] = pandas.to_numeric(full_data["sat_results"][c], errors='coerce', downcast='unsigned')
full_data['sat_results']['sat_score'] = full_data['sat_results'].iloc[:,-3:].sum(axis=1)
full_data['sat_results']['sat_score'] = full_data['sat_results']['sat_score'].astype('uint16')

In [8]:
full_data["sat_results"].dtypes

DBN                                object
SCHOOL NAME                        object
Num of SAT Test Takers             object
SAT Critical Reading Avg. Score    uint16
SAT Math Avg. Score                uint16
SAT Writing Avg. Score             uint16
sat_score                          uint16
dtype: object

In [9]:
full_data["hs_directory"]['lat'] = full_data["hs_directory"]['Location 1'].apply(lambda x: x.split("\n")[-1].replace("(", "").replace(")", "").split(", ")[0])
full_data["hs_directory"]['lon'] = full_data["hs_directory"]['Location 1'].apply(lambda x: x.split("\n")[-1].replace("(", "").replace(")", "").split(", ")[1])
for c in ['lat', 'lon']:
    full_data["hs_directory"][c] = full_data["hs_directory"][c].apply(pandas.to_numeric, errors='coerce')

In [10]:
full_data['ap_2010']=full_data['ap_2010'].drop([52],axis=0)

In [11]:
full_data['sat_results'].shape

(421, 7)

In [12]:
full_data["sat_results"]

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,sat_score
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355,404,363,1122
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383,423,366,1172
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377,402,370,1149
3,01M458,FORSYTH SATELLITE ACADEMY,7,414,401,359,1174
4,01M509,MARTA VALLE HIGH SCHOOL,44,390,433,384,1207
5,01M515,LOWER EAST SIDE PREPARATORY HIGH SCHOOL,112,332,557,316,1205
6,01M539,"NEW EXPLORATIONS INTO SCIENCE, TECHNOLOGY AND ...",159,522,574,525,1621
7,01M650,CASCADES HIGH SCHOOL,18,417,418,411,1246
8,01M696,BARD HIGH SCHOOL EARLY COLLEGE,130,624,604,628,1856
9,02M047,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECO...,16,395,400,387,1182


In [13]:
flat_data_names = [k for k,v in full_data.items()]
flat_data = [full_data[k] for k in flat_data_names]
full = flat_data[6]
# unique DBN check
for i, f in enumerate(flat_data[:]):
    name = flat_data_names[i]
    print(name)
    print(len(f["DBN"]) - len(f["DBN"].unique()))
# join type by iteration of 'name'
for i, f in enumerate(flat_data[:]):
    name = flat_data_names[i]
    #join_type = "left"
    #if name in ["ap_2010", "graduation"]:
        #join_type = "outer"
    if name not in ["math_test_results","sat_results"]:
        full = full.merge(f, on="DBN", how="left")

ap_2010
0
class_size
0
demographics
0
graduation
0
hs_directory
0
math_test_results
0
sat_results
0
survey
0


In [14]:
full.shape

(421, 166)

In [23]:
full.isnull().sum()

DBN                                       0
SCHOOL NAME                               0
Num of SAT Test Takers                    0
SAT Critical Reading Avg. Score           0
SAT Math Avg. Score                       0
SAT Writing Avg. Score                    0
sat_score                                 0
SchoolName                              170
AP Test Takers                          191
Total Exams Taken                       191
Number of Exams with scores 3 4 or 5    270
CSD                                      20
NUMBER OF STUDENTS / SEATS FILLED        20
NUMBER OF SECTIONS                       20
AVERAGE CLASS SIZE                       20
SIZE OF SMALLEST CLASS                   20
SIZE OF LARGEST CLASS                    20
SCHOOLWIDE PUPIL-TEACHER RATIO          421
Name                                      9
schoolyear                                9
fl_percent                              421
frl_percent                               9
total_enrollment                

In [37]:
df11=full[['DBN','SCHOOL NAME','SchoolName','AP Test Takers ']]
df11[df11.isna().any(axis=1)]

,DBN,SCHOOL NAME,SchoolName,AP Test Takers
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,NaN,NaN
3,01M458,FORSYTH SATELLITE ACADEMY,NaN,NaN
4,01M509,MARTA VALLE HIGH SCHOOL,NaN,NaN
7,01M650,CASCADES HIGH SCHOOL,NaN,NaN
8,01M696,BARD HIGH SCHOOL EARLY COLLEGE,NaN,NaN
9,02M047,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECO...,NaN,NaN
10,02M288,FOOD AND FINANCE HIGH SCHOOL,NaN,NaN
11,02M294,ESSEX STREET ACADEMY,NaN,NaN
12,02M296,HIGH SCHOOL OF HOSPITALITY MANAGEMENT,High School of Hospitality Management,NaN
17,02M308,LOWER MANHATTAN ARTS ACADEMY,Lower Manhattan Arts Academy,NaN


In [ ]:
full_data['sat_results']['DBN']

In [ ]:
flat_data[6]

In [ ]:
list(full_data['graduation'])

In [ ]:
df11.isna().sum()

In [ ]:
flat_data[6]['']

In [16]:
pandas.__version__

'0.24.2'

In [24]:
full_data['ap_2010']

,DBN,SchoolName,AP Test Takers,Total Exams Taken,Number of Exams with scores 3 4 or 5
0,01M448,UNIVERSITY NEIGHBORHOOD H.S.,39.0,49.0,10.0
1,01M450,EAST SIDE COMMUNITY HS,19.0,21.0,NaN
2,01M515,LOWER EASTSIDE PREP,24.0,26.0,24.0
3,01M539,"NEW EXPLORATIONS SCI,TECH,MATH",255.0,377.0,191.0
4,02M296,High School of Hospitality Management,NaN,NaN,NaN
5,02M298,Pace High School,21.0,21.0,NaN
6,02M300,"Urban Assembly School of Design and Construction,",99.0,117.0,10.0
7,02M303,"Facing History School, The",42.0,44.0,NaN
8,02M305,"Urban Assembly Academy of Government and Law, The",25.0,37.0,15.0
9,02M308,Lower Manhattan Arts Academy,NaN,NaN,NaN


In [41]:
full.to_csv(path_or_buf='full.csv')